In [1]:
import pandas as pd
import pickle

train_split = pd.read_csv(r'train_ndf.csv')
eval_split = pd.read_csv(r'val_ndf.csv')
ndf = pd.read_csv(r'ndf.csv')
test_splits = pd.read_csv(r'test_ndf.csv')

C:\Users\I6240624\AppData\Local\Temp\ipykernel_4076\1393575756.py:6: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  ndf = pd.read_csv(r'ndf.csv')


In [2]:
ndf

,Unnamed: 0,id,id.1,comments_disabled,is_video,taken_at_timestamp,edge_media_preview_like.count,edge_media_to_caption.edges,edge_media_to_comment.count,owner.id_x,...,sponsored,pic,pic_count,hashtag_count,tag_count,Followers,Followees,Posts,Name_lower,Category
0,0,0,0,False,False,1.433431e+09,3390.0,"#Regram from @lucavezil: ""You know why I love ...",26.0,2.221066e+08,...,0.0,['1000013365102810365.jpg'],1,5,2,507558,132,1061,magnum ice cream,Restaurants
1,1,1,1,False,False,1.433434e+09,125.0,Our society today is so desensitized to pornog...,5.0,2.376349e+08,...,0.0,['1000043359370989848.jpg'],1,0,1,0,0,0,0,0
2,2,2,2,False,False,1.433436e+09,22652.0,brought out my cute lil @benefitcosmetics pass...,62.0,1.953844e+07,...,0.0,['1000055615134224232.jpg'],1,1,1,0,0,0,0,0
3,3,3,3,False,False,1.433439e+09,272.0,How about Sweet Potato Pear and Broccoli by @a...,28.0,1.431250e+09,...,0.0,['1000084729422320243.jpg'],1,0,1,0,0,0,0,0
4,4,4,4,False,False,1.433447e+09,7407.0,"In the end, we only regret the chances we didn...",33.0,2.582803e+08,...,0.0,['1000147795992383306.jpg'],1,1,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88597,88597,88597,88597,False,False,1.499964e+09,517.0,Same Same But Different 😎#liketkit @liketoknow...,59.0,1.050761e+09,...,0.0,['1558137050267749810.jpg'],1,1,1,0,0,0,0,0
88598,88598,88598,88598,False,False,1.499965e+09,1345.0,I'm in love with these Hi Voltage high waisted...,13.0,1.780142e+08,...,0.0,['1558137734424806870.jpg'],1,0,2,73242,598,1933,curvywordy,Creators & Celebrities
88599,88599,88599,88599,False,False,1.499965e+09,382.0,still getting used to my new @bedheadbytigi cu...,16.0,1.946089e+09,...,0.0,['1558140170955794497.jpg'],1,2,1,0,0,0,0,0
88600,88600,88600,88600,False,False,1.499965e+09,524.0,Another look at this eye makeup using the @urb...,52.0,2.653621e+08,...,0.0,['1558142151136991440.jpg'],1,16,7,0,0,0,0,0


In [3]:
!pip install emoji
!pip install transformers
!pip install datasets
!pip install evaluate


[notice] A new release of pip is available: 23.1.1 -> 23.1.2
[notice] To update, run: C:\Users\I6240624\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.1.1 -> 23.1.2
[notice] To update, run: C:\Users\I6240624\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.1.1 -> 23.1.2
[notice] To update, run: C:\Users\I6240624\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.1.1 -> 23.1.2
[notice] To update, run: C:\Users\I6240624\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [4]:
datasets_dict = {"train_split": train_split, "eval_split": eval_split,  "test_splits": test_splits, 'ndf':ndf}

In [5]:
import numpy as np
from sklearn.model_selection import KFold
import datasets
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased", force_download=False)

def tokenize_function(examples):
  return tokenizer(examples["text"], padding="max_length", truncation=True)

def transform_dataset(df):
  df["sponsored"] = df["sponsored"].astype(bool)
  df["edge_media_to_caption.edges"] = df["edge_media_to_caption.edges"].astype(str)
  if isinstance(df, pd.DataFrame):
    return datasets.Dataset.from_pandas(df[["edge_media_to_caption.edges", "sponsored"]].rename(columns={"edge_media_to_caption.edges": "text", "sponsored": "label"}))
  else:
    return [transform_dataset(k) for k in df]
  
def tokenize_dataset(df):
  if isinstance(df, list):
    return [tokenize_dataset(k) for k in df]
  else:
    return df.map(tokenize_function, batched=True)
    
hf_datasets = {k: transform_dataset(v) for k, v in datasets_dict.items()}
tokenized_datasets = {k: tokenize_dataset(v) for k, v in hf_datasets.items()}

# hf_train = Dataset.from_pandas(task_train[["caption_task", "has_disclosures", "country", "size", "date"]].rename(columns={"caption_task": "text", "has_disclosures": "label"}))
# hf_test = Dataset.from_pandas(task_test[["caption_task", "has_disclosures", "country", "size", "date"]].rename(columns={"caption_task": "text", "has_disclosures": "label"}))
# hf_gold = Dataset.from_pandas(gold_data[["caption_task", "label", "country", "size", "year"]].rename(columns={"caption_task": "text"}))

C:\Users\I6240624\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
token=pd.DataFrame(tokenized_datasets['ndf'])
ndf=pd.merge(left=ndf, right=token, right_on='text', left_on='edge_media_to_caption.edges')

In [7]:
ndf

,Unnamed: 0,id,id.1,comments_disabled,is_video,taken_at_timestamp,edge_media_preview_like.count,edge_media_to_caption.edges,edge_media_to_comment.count,owner.id_x,...,Followers,Followees,Posts,Name_lower,Category,text,label,input_ids,token_type_ids,attention_mask
0,0,0,0,False,False,1.433431e+09,3390.0,"#Regram from @lucavezil: ""You know why I love ...",26.0,2.221066e+08,...,507558,132,1061,magnum ice cream,Restaurants,"#Regram from @lucavezil: ""You know why I love ...",False,"[101, 1001, 19723, 6444, 2013, 1030, 15604, 26...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
1,1,1,1,False,False,1.433434e+09,125.0,Our society today is so desensitized to pornog...,5.0,2.376349e+08,...,0,0,0,0,0,Our society today is so desensitized to pornog...,False,"[101, 2256, 2554, 2651, 2003, 2061, 4078, 6132...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
2,2,2,2,False,False,1.433436e+09,22652.0,brought out my cute lil @benefitcosmetics pass...,62.0,1.953844e+07,...,0,0,0,0,0,brought out my cute lil @benefitcosmetics pass...,False,"[101, 2716, 2041, 2026, 10140, 13451, 1030, 57...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
3,3,3,3,False,False,1.433439e+09,272.0,How about Sweet Potato Pear and Broccoli by @a...,28.0,1.431250e+09,...,0,0,0,0,0,How about Sweet Potato Pear and Broccoli by @a...,False,"[101, 2129, 2055, 4086, 14557, 28253, 1998, 22...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
4,4,4,4,False,False,1.433447e+09,7407.0,"In the end, we only regret the chances we didn...",33.0,2.582803e+08,...,0,0,0,0,0,"In the end, we only regret the chances we didn...",False,"[101, 1999, 1996, 2203, 1010, 2057, 2069, 9038...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
976349,88597,88597,88597,False,False,1.499964e+09,517.0,Same Same But Different 😎#liketkit @liketoknow...,59.0,1.050761e+09,...,0,0,0,0,0,Same Same But Different 😎#liketkit @liketoknow...,False,"[101, 2168, 2168, 2021, 2367, 100, 1001, 2066,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
976350,88598,88598,88598,False,False,1.499965e+09,1345.0,I'm in love with these Hi Voltage high waisted...,13.0,1.780142e+08,...,73242,598,1933,curvywordy,Creators & Celebrities,I'm in love with these Hi Voltage high waisted...,False,"[101, 1045, 1005, 1049, 1999, 2293, 2007, 2122...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
976351,88599,88599,88599,False,False,1.499965e+09,382.0,still getting used to my new @bedheadbytigi cu...,16.0,1.946089e+09,...,0,0,0,0,0,still getting used to my new @bedheadbytigi cu...,False,"[101, 2145, 2893, 2109, 2000, 2026, 2047, 1030...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
976352,88600,88600,88600,False,False,1.499965e+09,524.0,Another look at this eye makeup using the @urb...,52.0,2.653621e+08,...,0,0,0,0,0,Another look at this eye makeup using the @urb...,False,"[101, 2178, 2298, 2012, 2023, 3239, 5789, 2478...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."


In [8]:
from transformers import TrainingArguments

training_args = TrainingArguments(output_dir="test_trainer")

In [9]:
import numpy as np
import evaluate

acc = evaluate.load("accuracy", average="macro")
p = evaluate.load("precision", average="macro")
r = evaluate.load("recall", average="macro")
f1 = evaluate.load("f1", average="macro")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    accuracy = acc.compute(predictions=predictions, references=labels)["accuracy"]
    precision = p.compute(predictions=predictions, references=labels)["precision"]
    recall = r.compute(predictions=predictions, references=labels)["recall"]
    f1m = f1.compute(predictions=predictions, references=labels)["f1"]
    return {"accuracy": accuracy, "precision": precision, "recall": recall, "f1": f1m}

In [10]:
import torch
from transformers import AutoModelForSequenceClassification
from transformers import TrainingArguments, Trainer

model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2, force_download=True)
training_args = TrainingArguments(num_train_epochs=3, output_dir="test_trainer", evaluation_strategy="epoch", logging_strategy="epoch",)
trainer = Trainer(
  model=model,
  args=training_args,
  train_dataset=tokenized_datasets["train_split"],
  eval_dataset=tokenized_datasets["eval_split"],
  compute_metrics=compute_metrics,
)
# trainer.add_callback(CustomCallback(trainer)) 
trainer.train()
res = trainer.evaluate()

C:\Users\I6240624\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\huggingface_hub\file_download.py:133: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\I6240624\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Some weights of the model checkpoint at bert-ba

KeyboardInterrupt: 

In [ ]:
res

{'eval_loss': 0.030014000833034515,
 'eval_accuracy': 0.9924812030075187,
 'eval_precision': 0.8,
 'eval_recall': 1.0,
 'eval_f1': 0.888888888888889,
 'eval_runtime': 4.5842,
 'eval_samples_per_second': 29.013,
 'eval_steps_per_second': 3.708,
 'epoch': 3.0}

In [ ]:
# model.save_pretrained("/content/drive/MyDrive/Colab Notebooks/Instagram Project/clf_experiments/bert-base-multilingual-uncased-ads-us-only")
# tokenizer.save_pretrained("/content/drive/MyDrive/Colab Notebooks/Instagram Project/clf_experiments/bert-base-multilingual-uncased-ads-us-only")
model.save_pretrained(f"/content/drive/MyDrive/Colab Notebooks/Instagram Adaptive/bert-2019")
tokenizer.save_pretrained(f"/content/drive/MyDrive/Colab Notebooks/Instagram Adaptive/bert-2019")

('/content/drive/MyDrive/Colab Notebooks/Instagram Adaptive/bert-2019/tokenizer_config.json',
 '/content/drive/MyDrive/Colab Notebooks/Instagram Adaptive/bert-2019/special_tokens_map.json',
 '/content/drive/MyDrive/Colab Notebooks/Instagram Adaptive/bert-2019/vocab.txt',
 '/content/drive/MyDrive/Colab Notebooks/Instagram Adaptive/bert-2019/added_tokens.json',
 '/content/drive/MyDrive/Colab Notebooks/Instagram Adaptive/bert-2019/tokenizer.json')

In [ ]:
trainer.compute_metrics = compute_metrics
trainer.evaluate()

{'eval_loss': 0.030014000833034515,
 'eval_accuracy': 0.9924812030075187,
 'eval_precision': 0.8,
 'eval_recall': 1.0,
 'eval_f1': 0.888888888888889,
 'eval_runtime': 4.5566,
 'eval_samples_per_second': 29.188,
 'eval_steps_per_second': 3.731,
 'epoch': 3.0}

In [ ]:
from scipy.special import softmax

def predict(text, model, tokenizer):
    encoded_input = tokenizer(text, return_tensors="pt", truncation=True, max_length=tokenizer.model_max_length).to("cuda")
    output = model(**encoded_input)
    scores = output[0][0].detach().cpu().numpy()
    scores = softmax(scores)
    return scores.argmax()
    # print(scores.argmax())

In [ ]:
predict("Love playing with make up ❤️ used @beautyba ad buy this tbt", model=model, tokenizer=tokenizer)

1

In [ ]:
count = 0
ndf['prediction_BERT'] = 0
while count<len(ndf):
  ndf['prediction_BERT'][count] = predict(ndf['edge_media_to_caption.edges'][count], model=model, tokenizer=tokenizer)
  count=count+1

<ipython-input-59-738191857e35>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ndf['prediction_BERT'][count] = predict(ndf['edge_media_to_caption.edges'][count], model=model, tokenizer=tokenizer)


In [ ]:
#ndf = ndf.to_csv('ndf.csv', index = True)